https://plot.ly/ipython-notebooks/

https://nbviewer.jupyter.org/gist/bollwyvl/8c484445cbc149325bdc27cfa2482a7f

https://github.com/jupyter/help/issues/54#issuecomment-236326113

In [ ]:
import plotly
import plotly.offline
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()


import numpy as np
import time

## Creating an interactive graph inside IPython notebook

In [ ]:
import plotly.offline
from plotly.graph_objs import *

trace0 = Scatter(
  x=[1, 2, 3, 4],
  y=[10, 15, 13, 17]
)
trace1 = Scatter(
  x=[1, 2, 3, 4],
  y=[16, 5, 11, 9]
)
data = Data([trace0, trace1])

In [ ]:
#py.iplot(data, filename = 'basic-line')
plotly.offline.iplot(data)

### my inspection

In [ ]:
config = {}
config['showLink'] = True
config['linkText'] = 'Export to plot.ly'

plot_html, plotdivid, width, height = plotly.offline.offline._plot_html(
    data, config, True, '100%', 525, True
)

In [ ]:
plot_html

In [ ]:
plotdivid

In [ ]:
A = plot_html[:512].split('<script type="text/javascript">')[-1]
A += plot_html[512:]
B = A[:-9] # get rid of </script>

In [ ]:
B

In [ ]:
B.split(str(plotdivid))

In [ ]:
str(plotdivid).join(B.split(str(plotdivid)))==B

# ipywidgets solution to execute JavaScript
This is good since the JS to display is on the fly and cleared.

Alternatively, each update() [in JavaScript] call in this solution is equivalent to

```obj = IPython.display.Javascript(...)
IPython.display.display(obj)```

In [ ]:
from ipywidgets import HTML
from traitlets import Unicode, Int

gid = '83e52ad0-c854-49a4-8336-4bc892888d79' 
# this is a hard-coded part that should be integrated later
# this gid is of the above <div> in which Plotly draw plots.

class UpdaterJS(HTML):
    _view_module = Unicode("widget-dynamicJS-exec").tag(sync=True)
    _view_name = Unicode("HelloView").tag(sync=True)
    value = Unicode("").tag(sync=True)

In [ ]:
import IPython.display
obj = IPython.display.Javascript(
        'require.undef("widget-dynamicJS-exec");'
        'define("widget-dynamicJS-exec", ["jupyter-js-widgets"], function(widget){'
            'var HelloView = widget.DOMWidgetView.extend({'
                'render: function(){'
                    'this.pagetitle = document.createElement("div");'
                    'this.pagetitle.appendChild(document.createElement("div"));' # dummy child, needed because update() removes a child
                    'this.el.appendChild(this.pagetitle);'
                '},'
                'update: function(){'
                    'var container = this.pagetitle;'
                    'container.removeChild(container.childNodes[0]);'

                    'var child = document.createElement("script");'
                    'child.setAttribute("type", "text/javascript");'
                    'child.textContent = this.model.get("value");'
                    'container.appendChild(child);'
                '}'
            '});'
            'return {HelloView: HelloView}'
        '})'
    )
IPython.display.display(obj)

In [ ]:
updater2 = UpdaterJS()
updater2

The following string is hard-coded and should be generated by calling a proper Plotly function. The gid in it should also be generated by a aproper call and stored as a variable after this proper call.

In [ ]:
updater2.value = ("""require(["plotly"], function(Plotly) {window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot(\"""" + gid + 
"""", [{"type": "scatter", "x": [1, 2, 3, 4], "y": [10, 15, 13, 189]}, {"type": "scatter", "x": [1, 2, 3, 4], "y": [16, 5, 11, 9]}], {}, {"showLink": true, "linkText": "Export to plot.ly"})});""")